### Define the modules

In [1]:
import Noise_ph as ns
import Explorer_wf as ewf
import random
import copy
import numpy as np
import scipy as sc
import scipy.signal as sg
import scipy.fftpack as fftpack 
from matplotlib import pyplot as plt

### Noise Explorer Module to select the Noise from the storage.

In [2]:
raw_data_available = ns.Noise('C:/Users/ce_ti/OneDrive/Escritorio/Thesis/Phenom/Noise/L1/', "L1")
Detector = "L1"
raw_data_L1 = raw_data_available.select_Data(0)
print(raw_data_L1.keys())
strain_data_L1= ns.Noise_strain(raw_data_L1).strain()
GPS_Start_L1, Duration_L1, strain_time_L1 =ns.Noise_strain(raw_data_L1).strain_time()

*******************************
The available sets of data are the following:
No.1 ['C:/Users/ce_ti/OneDrive/Escritorio/Thesis/Phenom/Noise/L1\\L-L1_GWOSC_O3b_16KHZ_R1-1256665088-4096.hdf5']
Chosen strain data segment:
C:/Users/ce_ti/OneDrive/Escritorio/Thesis/Phenom/Noise/L1\L-L1_GWOSC_O3b_16KHZ_R1-1256665088-4096.hdf5
*******************************
L1 Keys: <KeysViewHDF5 ['meta', 'quality', 'strain']>
*******************************
<KeysViewHDF5 ['meta', 'quality', 'strain']>
*******************************
Strain_signal: [ 2.18581530e-19  2.20534486e-19  2.17628242e-19 ... -7.47271276e-19
 -7.88403545e-19 -7.45705850e-19]
*******************************
GPS start time: 1256665088
Segment Duration (s): 4096
Strain_time: [1.25666509e+09 1.25666509e+09 1.25666509e+09 ... 1.25666918e+09
 1.25666918e+09 1.25666918e+09]


### WF explorer Module to select the Phen WF

In [3]:
## Phen Reader
Slow_Dir = 'C:/Users/ce_ti/OneDrive/Escritorio/Thesis/Phenom/Generator/Slow/'
Ph_raw_data = ewf.Phen(Slow_Dir,"Phen")
selected_Ph = Ph_raw_data.select_Data(0)


# Variables of the Phenomenological Waveform
h_phen = selected_Ph["H"]
h_time = selected_Ph["Time"]
h_new = copy.copy(h_phen)

*******************************
The available set of data are the following:
The set segments of data available are the following:
1010
*******************************
Chosen strain data segments:
C:/Users/ce_ti/OneDrive/Escritorio/Thesis/Phenom/Generator/Slow\Slow_run1H0_Slope2667_f0(124.52)_f1(3492.96).csv
Phen Keys: Index(['H', 'Time'], dtype='object')
*******************************


In [5]:
fs=16384
len_Segmento = fs * 16 
n = int(len(strain_time_L1)/len_Segmento )

### Injections without jitter.

In [ ]:
%%time

strain_new = copy.copy(strain_data_L1)
labels = []

for i in range(0,n):
    s = i*16*fs
    labels.append(s)
    #print(f"Bloque:{i}", s)
    for k in range(0,len(h_new)):
        strain_new[s + k] = h_new[k] + strain_data_L1[s + k] 

### Injections with jitter from (-2,2) sec.

In [6]:
%%time
strain_noise = copy.copy(strain_data_L1)   # Strain of the Noise detectors
#jit_stor = []                              # Jitter storage  
D = 256                                    # Maximum Number of partitions
jit_stor = np.zeros(D)                     # Initialization of Jitter storage of D-Dimension

factor =     abs(np.max(strain_data_L1)) - abs(np.min(strain_data_L1)) / 2 # Factor of "Resample"
h_phenRes = h_phen * factor                                                # Signal Resized to the maximum of the Amplitude Noise 
jit_stor[0] = random.randint(0,2*fs)                                       # Established the first value of the jitter that can only be from 0 to 2 seg.

db = True                                  # Booleant that will allowed to exit the while loop.
contador = 0                               # 
i=1                                        # Initialize the number of the iterator i in i=1.
while db==True:
    jit_stor[i] = random.randint(-2*fs,2*fs)                               # jitter valor from -(2,2) sec.
    jit_stor[i]= int(jit_stor[i-1] + len(h_phenRes) +16*fs + jit_stor[i] ) # The real segment that count the jitter with the time.
    #print(jit_stor[i])
    if jit_stor[i] >= len(strain_data_L1):                                 # 
        db = False
        jit_stor = jit_stor[jit_stor!=0]
    else:
        for j in range(0,len(h_phenRes)):
            k = int(jit_stor[i-1])
            strain_noise[k + j] =  strain_data_L1[k+j] + h_phenRes[j]
    i+=1


Wall time: 4.39 s


### Define the Functions of FFT,PSD and SNR

In [81]:
def fft_signal(h,t):
    dt = t[1] - t [0]
    
    # Fourier Transform
    h_f = dt*fftpack.fft(h)
    f   = fftpack.fftfreq(len(t),dt)
    
    # Shift frequencies
    f   =  fftpack.fftshift(f)
    h_f =  fftpack.fftshift(h_f)
    
    # The one sided fft
    n   = len(f)
    f   = f[int(n/2)+1:n-1]
    h_f = 2.0*h_f[int(n/2)+1:n-1]
    
    return f,h_f

def PSD(h,fs):    
    nfft = 4 * fs                                    # Use 4 seconds of data for each fourier transform
    noverlap = 1 * nfft / 2                          # The number of points of overlap between segments used in Welch averaging
    freq,psd = sg.welch(h,fs=fs,window = window, return_onesided =True)
    
    freq = np.array(freq).reshape(freq.shape[0],1)   # Vectorization of freq.
    psd  = np.array(psd).reshape(psd.shape[0],1)     # Vectorization of psd.
    return freq, psd

def SNR(h,t,fpsd,psd,factor):
    f1,fft = fft_signal(h,t)                         # Calculate the FFT.
    fft = abs(fft)                                   # Absolute valor of the fft
    
    fft = fft*factor                                 # Multiplicate by the factor  
    
    tck = sc.interpolate.splrep(fpsd,psd,s=0)        # To have the same dim, we interpolate
    psd = sc.interpolate.splev(f1,tck,der=0)         # and have the PSD with the same dim with the one sided frequency of the FFT.
    
    df = f1[2] - f1[1]                               # Differential of the frequency

    print(f"PSD: {psd}")                                
    SNRsq =  np.sum(((fft**2)*df)/psd)               # Calculate the integral from the doc:

    SNR = np.sqrt(SNRsq)                             # Calculate the SNR
    
    return SNR


In [82]:
# Vectorization of the Phenomenological signal
h_phen = np.array(h_phen).reshape(h_phen.shape[0],1)
h_time = np.array(h_time).reshape(h_time.shape[0],1)
f_h,h_f = fft_signal(h_phen,h_time)

In [ ]:
plt.plot(f_h, h_f)

In [83]:
%%time
# The psd from the Noise, with out injections
fs = 16384
f_psd,S_psd = PSD(strain_data_L1,fs)
print(f"PSD is: {S_psd}")
#plt.loglog(S_psd)

PSD is: [[7.86519670e-41]
 [2.80062501e-41]
 [6.37771147e-42]
 ...
 [5.66142552e-53]
 [1.04096595e-53]
 [3.10773968e-54]]
Wall time: 3.69 s


In [84]:
%%time
snr = SNR(h_phen,h_time,f_psd,S_psd,1e-21)
print(f"The SNR is: {snr}")
#snr = SNR(h_phen,h_time,freq,psd,1e-53)

PSD: [2.80062501e-41 6.37771147e-42 8.58761382e-40 ... 3.70034066e-53
 5.66142552e-53 1.04096595e-53]
The SNR is: 774.854951226716
Wall time: 67.5 ms
